In [50]:
pip install sympy

Note: you may need to restart the kernel to use updated packages.


## Simpy es un simulador de procesos

Se ha realizado sobre la atencion a cliente [1]

In [1]:
import simpy as sp
import random
import math

In [46]:
SEMILLA = 30
NUM_CAJERO = 1
TIEMPO_CORTE_MIN = 15
TIEMPO_CORTE_MAX = 30
T_LLEGADAS = 20
TIEMPO_SIMULACION = 120
TOT_CLIENTES = 5

te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

In [47]:
def atender(cliente):
    global dt  #Para poder acceder a la variable dt declarada anteriormente
    R = random.random()  # Obtiene un numero aleatorio y lo guarda en R
    tiempo = TIEMPO_CORTE_MAX - TIEMPO_CORTE_MIN  
    tiempo_atencion = TIEMPO_CORTE_MIN + (tiempo*R) # Distribucion uniforme
    yield env.timeout(tiempo_atencion) # deja correr el tiempo n minutos
    print(" \o/ Se ha atendido a %s en %.2f minutos" % (cliente,tiempo_atencion))
    dt = dt + tiempo_atencion # Acumula los tiempos de uso de la i


def cliente (env, name, personal ):
    global te
    global fin
    llega = env.now # Guarda el minuto de llegada del cliente
    print ("---> %s llego banco en minuto %.2f" % (name, llega))
    with personal.request() as request: # Espera su turno
        yield request # Obtiene turno
        pasa = env.now # Guarda el minuto cuado comienza a ser atendido
        espera = pasa - llega # Calcula el tiempo que espero
        te = te + espera # Acumula los tiempos de espera
        print ("**** %s pasa con el cajero en minuto %.2f habiendo esperado %.2f" % (name, pasa, espera))
        yield env.process(atender(name)) # Invoca al proceso atender
        deja = env.now #Guarda el minuto en que termina el proceso atender
        print ("<--- %s deja el banco en minuto %.2f" % (name, deja))
        fin = deja # Conserva globalmente el ultimo minuto de la simulacion
    
def principal (env, personal):
    llegada = 0
    i = 0
    for i in range(TOT_CLIENTES): # Para n clientes
        R = random.random()
        llegada = -T_LLEGADAS * math.log(R) # Distribucion exponencial
        yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
        i += 1
        env.process(cliente(env, 'Cliente %d' % i, personal)) 


In [48]:
print ("------------------- Bienvenido Simulacion Peluqueria ------------------")
random.seed (SEMILLA)  # Cualquier valor
env = sp.Environment() # Crea el objeto entorno de simulacion
personal = sp.Resource(env, NUM_PELUQUEROS) #Crea los recursos (peluqueros)
env.process(principal(env, personal)) #Invoca el proceso princial
env.run() #Inicia la simulacion

------------------- Bienvenido Simulacion Peluqueria ------------------
---> Cliente 1 llego banco en minuto 12.36
**** Cliente 1 pasa con el cajero en minuto 12.36 habiendo esperado 0.00
 \o/ Se ha atendido a Cliente 1 en 15.45 minutos
<--- Cliente 1 deja el banco en minuto 27.81
---> Cliente 2 llego banco en minuto 37.17
**** Cliente 2 pasa con el cajero en minuto 37.17 habiendo esperado 0.00
---> Cliente 3 llego banco en minuto 45.67
 \o/ Se ha atendido a Cliente 2 en 18.15 minutos
<--- Cliente 2 deja el banco en minuto 55.32
**** Cliente 3 pasa con el cajero en minuto 55.32 habiendo esperado 9.65
---> Cliente 4 llego banco en minuto 72.83
 \o/ Se ha atendido a Cliente 3 en 20.96 minutos
<--- Cliente 3 deja el banco en minuto 76.28
**** Cliente 4 pasa con el cajero en minuto 76.28 habiendo esperado 3.45
---> Cliente 5 llego banco en minuto 81.70
 \o/ Se ha atendido a Cliente 4 en 29.83 minutos
<--- Cliente 4 deja el banco en minuto 106.11
**** Cliente 5 pasa con el cajero en minuto 

Conclusion: se puede observar que a partir del tercer cliente existe un tiempo de espero

[1]URL: https://naps.com.mx/blog/simulacion-en-python-usando-simpy/